In [48]:
!pip install pytorch_lightning
!pip install csv-logger
!pip install lightning

In [49]:
from transformers import BertModel
from transformers import BertTokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
from torch import nn
import pytorch_lightning as pl
import torch
from lightning.pytorch.loggers import CSVLogger
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, AutoModelWithLMHead
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
def load_dataset():
  train = pd.read_csv("./sample_data/train.csv")
  val = pd.read_csv("./sample_data/val.csv")
  test = pd.read_csv("./sample_data/test.csv")
  return {'train': train, 'val': val, 'test': test}

# @title ##### **Loading and Analysing Emotion Data**
dataset = load_dataset()
combined_dataset = pd.concat([dataset['train'], dataset['val'], dataset['test']])

print(f"{'Training data':>16}: {len(dataset['train'])}\n{'Validation data':>16}: {len(dataset['val'])} \
      \n{'Testing data':>16}: {len(dataset['test'])}\n{'Combined dataset':>16}: {len(combined_dataset)}")

   Training data: 2100
 Validation data: 700       
    Testing data: 700
Combined dataset: 3500


In [51]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_length, datacolumn, labelcolumn):
        self.data = df
        self.texts = self.data[datacolumn].tolist()
        self.labels = self.data[labelcolumn].tolist()
        # tokenizer and necessary parameters
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return(len(self.texts))

    def __getitem__(self,idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
                                                text,
                                                max_length = self.max_length,
                                                add_special_tokens = True,
                                                padding = 'max_length',
                                                truncation = True,
                                                return_attention_mask = True,
                                                return_tensors = 'pt',)

        return {'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'label': torch.tensor(label),
                }

In [82]:
BERT_MODEL = BertModel.from_pretrained("bert-base-uncased")
BERT_TOKENIZER = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

class BERTClassifier(pl.LightningModule):
    def __init__(self, model, num_class, learning_rate, epoch, train_size):
        super(BERTClassifier, self).__init__()
        self.model = model
        self.emotion_index = EMOTION_INDEX
        self.num_class = num_class
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.train_size = train_size
        self.pre_classifier = nn.Linear(768, 768)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, self.num_class)


    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)
        pooler = outputs[0][:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        logits = self.classifier(pooler)

        return logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]

        logits = self(input_ids, attention_mask)
        _, preds = torch.max(logits, dim = 1)
        loss = nn.CrossEntropyLoss()(logits, labels)

        self.log('train_loss', loss, on_epoch=True)
        self.log('train_acc',(preds == labels).sum() / len(labels), on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]

        logits = self(input_ids, attention_mask)
        _, preds = torch.max(logits, dim = 1)
        loss = nn.CrossEntropyLoss()(logits, labels)

        self.log('val_loss', loss)
        self.log('val_acc', (preds == labels).sum() / len(labels))
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]
        logits = self(input_ids=input_ids,
                      attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(logits, labels)
        _, preds = torch.max(logits, dim = 1)

        self.log('test_loss', loss)
        self.log('test_acc', (preds == labels).sum() / len(labels))

    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=self.learning_rate)
        scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=0,
                num_training_steps=self.epoch*self.train_size)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

    def emo_classification(self, dataset):
        predictions = []
        for _d in dataset:
            input_ids = _d['input_ids'].unsqueeze(0).to(device)
            attention_mask = _d['attention_mask'].unsqueeze(0).to(device)
            logits = self(input_ids, attention_mask)
            _, pred = torch.max(logits, dim = 1)
            index= pred.cpu().numpy()[0]
            predictions.append(self.emotion_index[index])
        return predictions

In [53]:
class BERTDataModule(pl.LightningDataModule):
    def __init__(self, traindf, valdf, testdf, batch_size, tokenizer, max_length, datacolumn, labelcolumn):
        super().__init__()
        # dataset
        self.traindf = traindf
        self.valdf = valdf
        self.testdf = testdf
        self.datacolumn = datacolumn
        self.labelcolumn = labelcolumn
        # parameters
        self.batch_size = batch_size
        self.max_length = max_length
        # tokenizer
        self.tokenizer = tokenizer

    def setup(self, stage=None):
        self.train_dataset = BERTDataset( self.traindf, self.tokenizer, self.max_length, self.datacolumn, self.labelcolumn)
        self.val_dataset = BERTDataset( self.valdf, self.tokenizer, self.max_length, self.datacolumn, self.labelcolumn)
        self.test_dataset = BERTDataset( self.testdf, self.tokenizer, self.max_length, self.datacolumn, self.labelcolumn)

    def train_dataloader(self):
        return DataLoader( self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader( self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader( self.test_dataset, batch_size=self.batch_size)

In [74]:
# Parameters
CLASSES = 7
DATACOLUMN = 'text'
LABLECOLUMN = 'index'
EMOTION_INDEX = {0: 'anger',
                 1: 'disgust',
                 2: 'fear',
                 3: 'joy',
                 4: 'neutral',
                 5: 'sadness',
                 6: 'surprise'}

# cuda means GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# parameters
EPOCH = 4 # two rounds of training
CLASSES = 7 # emotion categories
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
TRAIN_SIZE = len(dataset['train'])
MAX_LENGTH = BERT_TOKENIZER.model_max_length #512

DATACOLUMN = 'text'
BERTLABLECOLUMN = 'index'

In [84]:
# Datamodule
BERTdata_module = BERTDataModule(traindf=dataset['train'],
                               valdf=dataset['val'],
                               testdf=dataset['test'],
                               tokenizer=BERT_TOKENIZER,
                               batch_size=BATCH_SIZE,
                               max_length=MAX_LENGTH,
                               datacolumn=DATACOLUMN,
                               labelcolumn=BERTLABLECOLUMN)
BERTdata_module.setup()

# Classifier
BERTcls = BERTClassifier(
    model=BERT_MODEL,
    num_class=CLASSES,
    learning_rate=LEARNING_RATE,
    epoch=EPOCH,
    train_size=TRAIN_SIZE).to(device)

# Unblock to print classifier structure
# BERTcls

In [57]:
# Setup save path for log file
logger = CSVLogger("./logs", name='BERT_exp', version=1)

# Using pytorch trainer for tuning
trainer = pl.Trainer(
    accelerator="auto",
    devices="auto",
    max_epochs=EPOCH,
    precision="16-mixed",
    logger=logger)

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [58]:
trainer.fit(BERTcls, BERTdata_module)

/usr/local/lib/python3.10/dist-packages/lightning/fabric/loggers/csv_logs.py:269: Experiment logs directory ./logs/BERT_exp/version_1 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory ./logs/BERT_exp/version_1/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type      | Params | Mode 
-----------------------------------------------------
0 | model          | BertModel | 109 M  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=4` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=4` reached.


In [61]:
trainer.test(BERTcls, BERTdata_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7628571391105652     │
│         test_loss         │     0.779837965965271     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.779837965965271, 'test_acc': 0.7628571391105652}]

In [78]:
cleaned_data = pd.read_csv("./sample_data/format-not-clean.csv", sep=';')
cleaned_data['emotion'] = ''
cleaned_data['index'] = 0
cleaned_data.rename(columns={'Caption':'text'}, inplace=True)
Bertdata = BERTDataset(df = cleaned_data[['text', 'emotion', 'index']],
                       tokenizer = BERT_TOKENIZER,
                       max_length = BERT_TOKENIZER.model_max_length,
                       datacolumn = DATACOLUMN,
                       labelcolumn = LABLECOLUMN)


In [86]:
prediction = BERTcls.emo_classification(Bertdata)
cleaned_data['prediction'] = prediction

In [105]:
results = {'2020':[], '2021':[], '2022':[], '2023':[],}
results['2020'] = cleaned_data[cleaned_data['Date'] == 2020]['prediction'].tolist()
results['2021'] = cleaned_data[cleaned_data['Date'] == 2021]['prediction'].tolist()
results['2022'] = cleaned_data[cleaned_data['Date'] == 2022]['prediction'].tolist()
results['2023'] = cleaned_data[cleaned_data['Date'] == 2023]['prediction'].tolist()

In [109]:
import json
with open('results.json', 'w') as f:
    json.dump(results, f)